In [ ]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master/')
import timm

In [ ]:
from timm import create_model 
from fastai.vision.all import *

set_seed(42, reproducible=True)

In [ ]:
!tree ../input -d

In [ ]:
train_imgs_path = '../input/petfinder-pawpularity-score/train/'
train_df = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')

In [ ]:
def append_path(data, train=True):
    if train:
        return f'{train_imgs_path}{data}.jpg'
    else:
        return f'{test_imgs_path}{data}.jpg'

In [ ]:
train_df['full_path'] = train_df['Id'].apply(lambda x: append_path(x))
train_df['norm_score'] = train_df['Pawpularity']/100
train_df.head(5)

In [ ]:
dls = DataBlock(blocks=(ImageBlock, RegressionBlock),
                get_x=ColReader('full_path'),
                get_y=ColReader('norm_score'),
                splitter=RandomSplitter(0.2),
                item_tfms=Resize(450),
                batch_tfms=setup_aug_tfms([RandomResizedCrop(size=224), Flip(size=224)]),
               )

In [ ]:
paw_dls = dls.dataloaders(train_df , batch_size= 32, seed=12)
paw_dls.show_batch()

In [ ]:
len(paw_dls.train.dataset), len(paw_dls.valid.dataset)

In [ ]:
if not os.path.exists('/root/.cache/torch/hub/checkpoints/'):
    os.makedirs('/root/.cache/torch/hub/checkpoints/')
!cp '../input/swin-transformer/swin_large_patch4_window7_224_22kto1k.pth' '/root/.cache/torch/hub/checkpoints/swin_large_patch4_window7_224_22kto1k.pth'

In [ ]:
model = create_model('swin_large_patch4_window7_224', 
                     pretrained=True, 
                     num_classes=paw_dls.c)

In [ ]:
def metric_rmse(input,target):
    return 100*torch.sqrt(F.mse_loss(F.sigmoid(input.flatten()), target))

In [ ]:
learn = Learner(paw_dls, 
                model, 
                loss_func = BCEWithLogitsLossFlat(), 
                metrics = metric_rmse, 
                cbs=[MixUp(0.2)]).to_fp16()

In [ ]:
learn.lr_find(end_lr=1e-2)

In [ ]:
learn.fit_one_cycle(6, 2e-5, cbs=[EarlyStoppingCallback(monitor='metric_rmse', 
                                               comp=np.less, 
                                               patience=3)])

In [ ]:
learn.recorder.plot_loss()

In [ ]:
learn = learn.to_fp32()

In [ ]:
learn.save('pawpular_model')
learn.export()

In [ ]:
dataset_path = Path('../input/petfinder-pawpularity-score/')

In [ ]:
test_df = pd.read_csv(dataset_path/'test.csv')
test_df.head()

In [ ]:
test_df['Pawpularity'] = [1]*len(test_df)
test_df['full_path'] = test_df['Id'].map(lambda x:str(dataset_path/'test'/x)+'.jpg')
test_df = test_df.drop(columns=['Id'])
train_df['norm_score'] = train_df['Pawpularity']/100
test_dl = paw_dls.test_dl(test_df)

In [ ]:
test_dl.show_batch()

In [ ]:
preds, _ = learn.tta(dl=test_dl, n=10, beta=0)

In [ ]:
sample_df = pd.read_csv(dataset_path/'sample_submission.csv')
sample_df['Pawpularity'] = preds.float().numpy()*100
sample_df.to_csv('submission.csv',index=False)